---
title: "Machine Learning Reflective Essay"
theme:
  light: lux
sidebar: false
jupyter: python3
---

In [ ]:
'''
ML Project for Module:
BAA10127 - Data Analytics: Machine Learning & Advanced Python
Student No. 21311696
Student Name: Rory James Mulhern
Course: BSI4
Dataset: https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data
'''

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [ ]:
# Linking file to Code
filepath = '/Users/mulhr/Desktop/ML Project/loan_data.csv'

# Importing the dataset
loans_df = pd.read_csv(filepath)

loans_df

In [ ]:
# Youssef Elbadry Accessed: 9th April 2025

# Looking at info on the data
loans_df.info()

In [ ]:
# Changing the person_age column to an integer
loans_df['person_age'] = loans_df['person_age'].astype(int)

# Looking at info on the data
loans_df.info()

In [ ]:
# Removing duplicate rows
loans_df.drop_duplicates(inplace=True)

# Check if there are any duplicates left
duplicate_count = loans_df.duplicated().sum()

# Display final check
if duplicate_count == 0:
    print("No duplicate values in the dataset.")
else:
    print(f"Total duplicate values remaining: {duplicate_count}")

In [ ]:
# Looking at the data description see the statistics of numeric columns
loans_df.describe().T

In [ ]:
# Youssef Elbadry Accessed: 9th April 2025

# Seeing which columns are Categorical and Numerical
cat_cols = [var for var in loans_df.columns if loans_df[var].dtypes == 'object']
num_cols = [var for var in loans_df.columns if loans_df[var].dtypes != 'object']

print(f'Categorical columns: {cat_cols}')
print(f'Numerical columns: {num_cols}')

In [ ]:
cat_cols

In [ ]:
# Seeing the split in gender
loans_df['person_gender'].value_counts()

In [ ]:
# Youssef Elbadry Accessed: 9th April 2025
def plot_categorical_column(dataframe, column):

    plt.figure(figsize=(7, 7))
    ax = sns.countplot(x=dataframe[column])
    total_count = len(dataframe[column])
    threshold = 0.05 * total_count
    category_counts = dataframe[column].value_counts(normalize=True) * 100
    ax.axhline(threshold, color='red', linestyle='--', label=f'0.05% of total count ({threshold:.0f})')
    
    for p in ax.patches:
        height = p.get_height()
        percentage = (height / total_count) * 100
        ax.text(p.get_x() + p.get_width() / 2., height + 0.02 * total_count, f'{percentage:.2f}%', ha="center")
    
    plt.title(f'Label Cardinality for "{column}" Column')
    plt.ylabel('Count')
    plt.xlabel(column)
    plt.tight_layout()
    
    plt.legend()
    plt.show()

for col in cat_cols:
    plot_categorical_column(loans_df, col)

In [ ]:
loans_df[num_cols].hist(bins=30, figsize=(12,10))
plt.show()

In [ ]:
label_prop = loans_df['loan_status'].value_counts()

plt.pie(label_prop.values, labels=['Rejected (0)', 'Approved (1)'], autopct='%.2f')
plt.title('Target label proportions')
plt.show()

In [ ]:
'''
Article saying most lenders will not lend to anyone above 70
https://www.moneysupermarket.com/loans/loans-for-pensioners/#:~:text=Most%20lenders%20have%20a%20maximum,beyond%20this%20age%20is%20rare.
'''
loans_df = loans_df[loans_df['person_age']<= 70]
print('Ages above 70 removed!')

In [ ]:
loans_df[num_cols].hist(bins=30, figsize=(12,10))
plt.show()

In [ ]:
# Sulani Ishara Accessed: 14th April 2025
numerical_columns = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']

fig, axes = plt.subplots(4, 2, figsize=(16, 20))
fig.suptitle('Numerical Features vs Loan Status (Density Plots)', fontsize=16)

for i, col in enumerate(numerical_columns):
    sns.kdeplot(data=loans_df, x=col, hue='loan_status', ax=axes[i//2, i%2], fill=True, common_norm=False, palette='muted')
    axes[i//2, i%2].set_title(f'{col} vs Loan Status')
    axes[i//2, i%2].set_xlabel(col)
    axes[i//2, i%2].set_ylabel('Density')

fig.delaxes(axes[3, 1])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Box and Whisker plot to see what the outliers in the dataset look like
# Sulani Ishara Accessed: 14th April 2025

# Function to perform univariate analysis for numeric columns
def univariate_analysis(data, column, title):
    plt.figure(figsize=(10, 2))
    
    sns.boxplot(x=data[column], color='sandybrown')
    plt.title(f'{title} Boxplot')
    
    plt.tight_layout()
    plt.show()

    print(f'\nSummary Statistics for {title}:\n', data[column].describe())

columns_to_analyse = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']

for column in columns_to_analyse:
    univariate_analysis(loans_df, column, column.replace('_', ' '))

In [ ]:
from sklearn.preprocessing import RobustScaler
from scipy.stats.mstats import winsorize

for col in ["person_age", "person_income", "person_emp_exp", "loan_amnt"]:
    loans_df[col] = winsorize(loans_df[col], limits=[0.025, 0.025])
# Robust scaling
scaler = RobustScaler()
loans_df[["person_age", "person_income", "person_emp_exp", "loan_amnt"]] = scaler.fit_transform(loans_df[["person_age", "person_income", "person_emp_exp", "loan_amnt"]])

# Box and Whisker plot to see what the outliers in the dataset look like
# Function to perform univariate analysis for numeric columns

for column in columns_to_analyse:
    univariate_analysis(loans_df, column, column.replace('_', ' '))

In [ ]:
columns_to_check = ["person_age", "person_income", "person_emp_exp", "loan_amnt"]

for col in columns_to_check:
    skew_val = loans_df[col].skew()
    print(f"{col} skewness: {skew_val:.2f}")

In [ ]:
# Apply log1p directly — it's safe for 0s
for col in columns_to_check:
    loans_df[col] = np.log1p(loans_df[col])

# Recheck skewness
for col in columns_to_check:
    skew_val = loans_df[col].skew()
    print(f"{col} skewness after log1p: {skew_val:.2f}")

for column in columns_to_analyse:
    univariate_analysis(loans_df, column, column.replace('_', ' '))

In [ ]:
loans_df
loans_df.describe().T

In [ ]:
# Sulani Ishara Accessed: 14th April 2025
numerical_columns = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']

fig, axes = plt.subplots(4, 2, figsize=(16, 20))
fig.suptitle('Numerical Features vs Loan Status (Density Plots)', fontsize=16)

for i, col in enumerate(numerical_columns):
    sns.kdeplot(data=loans_df, x=col, hue='loan_status', ax=axes[i//2, i%2], fill=True, common_norm=False, palette='muted')
    axes[i//2, i%2].set_title(f'{col} vs Loan Status')
    axes[i//2, i%2].set_xlabel(col)
    axes[i//2, i%2].set_ylabel('Density')

fig.delaxes(axes[3, 1])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Making Education into a non-categorical columns
loans_df['person_education'] = loans_df['person_education'].replace({
    'High School': 0,
    'Associate': 1,
    'Bachelor': 2,
    'Master': 3,
    'Doctorate': 4
})

In [ ]:
loans_df

In [ ]:
# One-hot coding for dummy variables
loans_df = pd.get_dummies(loans_df, columns = ['person_gender', 'person_home_ownership', 'loan_intent', 'previous_loan_defaults_on_file'], drop_first = True)

# Checking the data types
loans_df.dtypes

In [ ]:
# Define numerical columns with target
numerical_columns_with_target = [
    'person_age', 
    'person_income', 
    'person_emp_exp', 
    'loan_amnt', 
    'loan_int_rate', 
    'loan_percent_income', 
    'cb_person_cred_hist_length', 
    'credit_score'
]

# Create pairplot of numerical features with loan_status as hue
sns.pairplot(loans_df[numerical_columns_with_target + ['loan_status']], 
             hue='loan_status', 
             palette='muted'
            )
plt.show()

In [ ]:
# Getting a correlation matrix
num_loans_df = loans_df.select_dtypes(include=['number']) # Include only numerical data types

# Correlation of that data
corr_matrix = num_loans_df.corr()
print(corr_matrix)

In [ ]:
# Visual the Correlation Matrix
plt.figure(figsize=(16, 12))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix of Variables')
plt.show()

In [ ]:
# Drop Person Employment Experience and Age
loans_df = loans_df.drop(columns=['person_emp_exp','person_age'])
loans_df

In [ ]:
# Create a new column for custom labels
loans_df['loan_status_label'] = loans_df['loan_status'].map({0: 'Denied (0)', 1: 'Approved (1)'})

# Create a histogram plotting Approved and Denied loans
sns.histplot(
    data=loans_df,
    x='loan_status_label',
    hue='loan_status_label',
    palette={"Denied (0)": "red", "Approved (1)": "green"}
)
plt.title("Amount of Denied and Approved Loans")
plt.xlabel("Loan Status")
plt.ylabel("Count")
plt.show

In [ ]:
# Splitting the Dataset into X and Y
X = loans_df.drop(columns=['loan_status', 'loan_status_label'])  
y = loans_df['loan_status'] 

In [ ]:
X

In [ ]:
y

In [ ]:
# Splitting the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
lr_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    reg_model_lr = LogisticRegression(max_iter=200000, random_state=42)
    reg_model_lr.fit(X_resampled, y_resampled)
    
    # Evaluate the model on the test data
    lr_accuracy = reg_model_lr.score(X_test, y_test)
    lr_accuracy_scores.append(lr_accuracy)
    print(f"Fold {fold} Accuracy: {lr_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(lr_accuracy_scores)/len(lr_accuracy_scores):.4f}")

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
lr2_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    reg_model_lr2 = LogisticRegression(max_iter=200000, random_state=42, penalty='l2')
    reg_model_lr2.fit(X_resampled, y_resampled)
    
    # Evaluate the model on the test data
    lr2_accuracy = reg_model_lr2.score(X_test, y_test)
    lr2_accuracy_scores.append(lr2_accuracy)
    print(f"Fold {fold} Accuracy: {lr2_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(lr2_accuracy_scores)/len(lr2_accuracy_scores):.4f}")

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_lr = reg_model_lr.predict(X_test)

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_lr2 = reg_model_lr2.predict(X_test)

In [ ]:
# Compute the evaluation metrics
lr_precision = precision_score(y_test, predictions_lr)
lr_recall = recall_score (y_test, predictions_lr)
lr_f1 = f1_score(y_test, predictions_lr)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(lr_accuracy_scores)/len(lr_accuracy_scores):.4f}")
print(f"Precision: {lr_precision:.4f}")
print(f"Recall: {lr_recall:.4f}")
print(f"F1-Score: {lr_f1:.4f}")

In [ ]:
# Compute the evaluation metrics
lr2_precision = precision_score(y_test, predictions_lr2)
lr2_recall = recall_score (y_test, predictions_lr2)
lr2_f1 = f1_score(y_test, predictions_lr2)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(lr2_accuracy_scores)/len(lr2_accuracy_scores):.4f}")
print(f"Precision: {lr2_precision:.4f}")
print(f"Recall: {lr2_recall:.4f}")
print(f"F1-Score: {lr2_f1:.4f}")

In [ ]:
lr_cm = confusion_matrix(y_test, predictions_lr )
print(lr_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(lr_cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (Logistic Regression)", fontsize=14)
plt.show()

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
lr_y_prob = reg_model_lr.predict_proba(X_test)[:, 1]

lr_auc_roc = roc_auc_score(y_test, lr_y_prob)
print(f"AUC-ROC: {lr_auc_roc:.4f}")

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
lr2_y_prob = reg_model_lr2.predict_proba(X_test)[:, 1]

lr2_auc_roc = roc_auc_score(y_test, lr2_y_prob)
print(f"AUC-ROC: {lr2_auc_roc:.4f}")

In [ ]:
# From ChatGPT

# Get false positive rate, true positive rate and thresholds
fpr, tpr, thresholds = roc_curve(y_test, lr_y_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {lr_auc_roc:.4f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line for random classifier
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (Logistic Regression)')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
dt_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_resampled, y_resampled)
    
    # Evaluate the model on the test data
    dt_accuracy = dt_model.score(X_test, y_test)
    dt_accuracy_scores.append(dt_accuracy)
    print(f"Fold {fold} Accuracy: {dt_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(dt_accuracy_scores)/len(dt_accuracy_scores):.4f}")

In [ ]:
# Getting the predictions for the Decision Tree Model
predictions_dt = dt_model.predict(X_test)

In [ ]:
# Compute the evaluation metrics
dt_precision = precision_score(y_test, predictions_dt)
dt_recall = recall_score (y_test, predictions_dt)
dt_f1 = f1_score(y_test, predictions_dt)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(dt_accuracy_scores)/len(dt_accuracy_scores):.4f}")
print(f"Precision: {dt_precision:.4f}")
print(f"Recall: {dt_recall:.4f}")
print(f"F1-Score: {dt_f1:.4f}")

In [ ]:
dt_cm = confusion_matrix(y_test, predictions_dt )
print(dt_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(dt_cm, annot=True, fmt="d", cmap="Blues",  cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (Decision Tree)", fontsize=14)
plt.show()

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
dt_y_prob = dt_model.predict_proba(X_test)[:, 1]

dt_auc_roc = roc_auc_score(y_test, dt_y_prob)
print(f"AUC-ROC: {dt_auc_roc:.4f}")

In [ ]:
# From ChatGPT

# Get false positive rate, true positive rate and thresholds
fpr, tpr, thresholds = roc_curve(y_test, dt_y_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {dt_auc_roc:.4f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line for random classifier
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (Decsision Tree)')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
rf_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_resampled, y_resampled)
    
    # Evaluate the model on the test data
    rf_accuracy = rf_model.score(X_test, y_test)
    rf_accuracy_scores.append(rf_accuracy)
    print(f"Fold {fold} Accuracy: {rf_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(rf_accuracy_scores)/len(rf_accuracy_scores):.4f}")

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
rf2_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    rf2_model = RandomForestClassifier(n_estimators=200, 
                                       random_state=42, 
                                       max_depth=8,
                                       min_samples_split=5,
                                       min_samples_leaf=2,
                                       max_features='sqrt',
                                       bootstrap=True)
    rf2_model.fit(X_resampled, y_resampled)
    
    # Evaluate the model on the test data
    rf2_accuracy = rf2_model.score(X_test, y_test)
    rf2_accuracy_scores.append(rf2_accuracy)
    print(f"Fold {fold} Accuracy: {rf2_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(rf2_accuracy_scores)/len(rf2_accuracy_scores):.4f}")

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
rf3_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    rf3_model = RandomForestClassifier(n_estimators=200, 
                                       random_state=42, 
                                       max_depth=8,
                                       min_samples_split=5,
                                       min_samples_leaf=2,
                                       max_features='sqrt',
                                       bootstrap=False)
    rf3_model.fit(X_resampled, y_resampled)
    
    # Evaluate the model on the test data
    rf3_accuracy = rf3_model.score(X_test, y_test)
    rf3_accuracy_scores.append(rf3_accuracy)
    print(f"Fold {fold} Accuracy: {rf3_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(rf3_accuracy_scores)/len(rf3_accuracy_scores):.4f}")

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_rf = rf_model.predict(X_test)

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_rf2 = rf2_model.predict(X_test)

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_rf3 = rf3_model.predict(X_test)

In [ ]:
# Compute the evaluation metrics
rf_precision = precision_score(y_test, predictions_rf)
rf_recall = recall_score (y_test, predictions_rf)
rf_f1 = f1_score(y_test, predictions_rf)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(rf_accuracy_scores)/len(rf_accuracy_scores):.4f}")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall: {rf_recall:.4f}")
print(f"F1-Score: {rf_f1:.4f}")

In [ ]:
# Compute the evaluation metrics
rf2_precision = precision_score(y_test, predictions_rf2)
rf2_recall = recall_score (y_test, predictions_rf2)
rf2_f1 = f1_score(y_test, predictions_rf2)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(rf2_accuracy_scores)/len(rf2_accuracy_scores):.4f}")
print(f"Precision: {rf2_precision:.4f}")
print(f"Recall: {rf2_recall:.4f}")
print(f"F1-Score: {rf2_f1:.4f}")

In [ ]:
# Compute the evaluation metrics
rf3_precision = precision_score(y_test, predictions_rf3)
rf3_recall = recall_score (y_test, predictions_rf3)
rf3_f1 = f1_score(y_test, predictions_rf3)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(rf3_accuracy_scores)/len(rf3_accuracy_scores):.4f}")
print(f"Precision: {rf3_precision:.4f}")
print(f"Recall: {rf3_recall:.4f}")
print(f"F1-Score: {rf3_f1:.4f}")

In [ ]:
rf_cm = confusion_matrix(y_test, predictions_rf)
print(rf_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(rf_cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (Random Forest (Untuned))", fontsize=14)
plt.show()

In [ ]:
rf2_cm = confusion_matrix(y_test, predictions_rf2)
print(rf2_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(rf2_cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (Random Forest (Tuned v1))", fontsize=14)
plt.show()

In [ ]:
rf3_cm = confusion_matrix(y_test, predictions_rf2)
print(rf3_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(rf3_cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (Random Forest (Tuned v2))", fontsize=14)
plt.show()

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
rf_y_prob = rf_model.predict_proba(X_test)[:, 1]

rf_auc_roc = roc_auc_score(y_test, rf_y_prob)
print(f"AUC-ROC: {rf_auc_roc:.4f}")

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
rf2_y_prob = rf2_model.predict_proba(X_test)[:, 1]

rf2_auc_roc = roc_auc_score(y_test, rf2_y_prob)
print(f"AUC-ROC: {rf2_auc_roc:.4f}")

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
rf3_y_prob = rf3_model.predict_proba(X_test)[:, 1]

rf3_auc_roc = roc_auc_score(y_test, rf3_y_prob)
print(f"AUC-ROC: {rf3_auc_roc:.4f}")

In [ ]:
# From ChatGPT

# Get false positive rate, true positive rate and thresholds
fpr, tpr, thresholds = roc_curve(y_test, rf_y_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {rf_auc_roc:.4f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line for random classifier
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (Random Forest (Untuned))')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Dictionary of model names and predicted probabilities
models_probs = {
    "Random Forest(Untuned)": rf_y_prob,
    "Random Forest(Tuned v1)": rf2_y_prob,
    "Random Forest(Tuned v2)": rf3_y_prob
}

plt.figure(figsize=(10, 8))

# Plot each ROC curve
for name, probs in models_probs.items():
    fpr, tpr, _ = roc_curve(y_test, probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.5f})')

# Plot random guess line
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison of Models (Random Forest)')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
xgb_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    xgb_model = XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        eval_metric='logloss',
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    
    # Evaluate the model on the test data
    xgb_accuracy = xgb_model.score(X_test, y_test)
    xgb_accuracy_scores.append(xgb_accuracy)
    print(f"Fold {fold} Accuracy: {xgb_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(xgb_accuracy_scores)/len(xgb_accuracy_scores):.4f}")

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
xgb2_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    xgb2_model = XGBClassifier(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.1,
        eval_metric='logloss',
        random_state=42
    )
    xgb2_model.fit(X_train, y_train)
    
    # Evaluate the model on the test data
    xgb2_accuracy = xgb2_model.score(X_test, y_test)
    xgb2_accuracy_scores.append(xgb2_accuracy)
    print(f"Fold {fold} Accuracy: {xgb2_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(xgb2_accuracy_scores)/len(xgb2_accuracy_scores):.4f}")

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_xgb = xgb_model.predict(X_test)

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_xgb2 = xgb2_model.predict(X_test)

In [ ]:
# Compute the evaluation metrics
xgb_precision = precision_score(y_test, predictions_xgb)
xgb_recall = recall_score (y_test, predictions_xgb)
xgb_f1 = f1_score(y_test, predictions_xgb)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(xgb_accuracy_scores)/len(xgb_accuracy_scores):.4f}")
print(f"Precision: {xgb_precision:.4f}")
print(f"Recall: {xgb_recall:.4f}")
print(f"F1-Score: {xgb_f1:.4f}")

In [ ]:
# Compute the evaluation metrics
xgb2_precision = precision_score(y_test, predictions_xgb2)
xgb2_recall = recall_score (y_test, predictions_xgb2)
xgb2_f1 = f1_score(y_test, predictions_xgb2)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(xgb2_accuracy_scores)/len(xgb2_accuracy_scores):.4f}")
print(f"Precision: {xgb2_precision:.4f}")
print(f"Recall: {xgb2_recall:.4f}")
print(f"F1-Score: {xgb2_f1:.4f}")

In [ ]:
xgb_cm = confusion_matrix(y_test, predictions_xgb)
print(xgb_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(xgb_cm, annot=True, fmt="d", cmap="Blues",  cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (XGBoost (Untuned))", fontsize=14)
plt.show()

In [ ]:
xgb2_cm = confusion_matrix(y_test, predictions_xgb2)
print(xgb2_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(xgb2_cm, annot=True, fmt="d", cmap="Blues",  cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (XGBoost (Tuned))", fontsize=14)
plt.show()

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
xgb_y_prob = xgb_model.predict_proba(X_test)[:, 1]

xgb_auc_roc = roc_auc_score(y_test, xgb_y_prob)
print(f"AUC-ROC: {xgb_auc_roc:.4f}")

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
xgb2_y_prob = xgb2_model.predict_proba(X_test)[:, 1]

xgb2_auc_roc = roc_auc_score(y_test, xgb2_y_prob)
print(f"AUC-ROC: {xgb2_auc_roc:.4f}")

In [ ]:
# From ChatGPT

# Get false positive rate, true positive rate and thresholds
fpr, tpr, thresholds = roc_curve(y_test, xgb_y_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {xgb_auc_roc:.4f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line for random classifier
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (XGBoost (Untuned))')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Dictionary of model names and predicted probabilities
models_probs = {
    "XGBoost (Tuned)": xgb2_y_prob,
    "XGBoost (Untuned)": xgb_y_prob,
}

plt.figure(figsize=(10, 8))

# Plot each ROC curve
for name, probs in models_probs.items():
    fpr, tpr, _ = roc_curve(y_test, probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.5f})')

# Plot random guess line
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison of Models (XGBoost)')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Setting Up 10-Fold Stratified Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
knn_accuracy_scores = []

# Loop through each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_resampled, X_test = X.iloc[train_index], X.iloc[test_index]
    y_resampled, y_test = y.iloc[train_index], y.iloc[test_index]

    # --- Model Training ---
    knn_model = KNeighborsClassifier(
        n_neighbors=2, 
        weights='uniform', 
        algorithm='auto', 
        leaf_size=30, 
        metric='minkowski'
    )
    knn_model.fit(X_train, y_train)
    
    # Evaluate the model on the test data
    knn_accuracy = knn_model.score(X_test, y_test)
    knn_accuracy_scores.append(knn_accuracy)
    print(f"Fold {fold} Accuracy: {knn_accuracy:.4f}")
    
print(f"Average Accuracy: {sum(knn_accuracy_scores)/len(knn_accuracy_scores):.4f}")

In [ ]:
# Getting the predictions for the Logistic Regression Model
predictions_knn = knn_model.predict(X_test)

In [ ]:
# Compute the evaluation metrics
knn_precision = precision_score(y_test, predictions_knn)
knn_recall = recall_score (y_test, predictions_knn)
knn_f1 = f1_score(y_test, predictions_knn)

# Print out evaluation metrics
print(f"Average Accuracy: {sum(knn_accuracy_scores)/len(xgb_accuracy_scores):.4f}")
print(f"Precision: {knn_precision:.4f}")
print(f"Recall: {knn_recall:.4f}")
print(f"F1-Score: {knn_f1:.4f}")

In [ ]:
knn_cm = confusion_matrix(y_test, predictions_knn)
print(knn_cm)

# Define new labels: index 0 -> "Denied", index 1 -> "Approved"
labels = ['Denied', 'Approved']

# Plot the confusion matrix heatmap with the renamed labels
plt.figure(figsize=(8, 6))
sns.heatmap(knn_cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Predicted Denied", "Predicted Approved"],
            yticklabels=["Actual Denied", "Actual Approved"])
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title("Confusion Matrix (KNN)", fontsize=14)
plt.show()

In [ ]:
# Calculating the AUC-ROC | from one of the tutorials
knn_y_prob = knn_model.predict_proba(X_test)[:, 1]

knn_auc_roc = roc_auc_score(y_test, knn_y_prob)
print(f"AUC-ROC: {knn_auc_roc:.4f}")

In [ ]:
# Get false positive rate, true positive rate and thresholds
fpr, tpr, thresholds = roc_curve(y_test, knn_y_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {knn_auc_roc:.4f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line for random classifier
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (KNN)')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Dictionary of model names and predicted probabilities
models_probs = {
    "Logistic Regression": lr_y_prob,
    "Decision Tree": dt_y_prob,
    "Random Forest": rf_y_prob,
    "Random Forest(Tuned v1)": rf2_y_prob,
    "Random Forest(Tuned v2)": rf3_y_prob,
    "XGBoost (Tuned)": xgb2_y_prob,
    "XGBoost (Untuned)": xgb_y_prob,
    "KNN": knn_y_prob

}

plt.figure(figsize=(10, 8))

# Plot each ROC curve
for name, probs in models_probs.items():
    fpr, tpr, _ = roc_curve(y_test, probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})')

# Plot random guess line
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison of Models (All Models)')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Print out all evaluation metrics
print("Logistic Regression (Untuned) Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(lr_accuracy_scores)/len(lr_accuracy_scores):.4f}")
print(f"Precision: {lr_precision:.4f}")
print(f"Recall: {lr_recall:.4f}")
print(f"F1-Score: {lr_f1:.4f}")
print(f"AUC-ROC: {lr_auc_roc:.4f}")

print("  ")
print("Logistic Regression (Tuned) Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(lr2_accuracy_scores)/len(lr2_accuracy_scores):.4f}")
print(f"Precision: {lr2_precision:.4f}")
print(f"Recall: {lr2_recall:.4f}")
print(f"F1-Score: {lr2_f1:.4f}")
print(f"AUC-ROC: {lr2_auc_roc:.4f}")

print("  ")
print("Decision Tree Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(dt_accuracy_scores)/len(dt_accuracy_scores):.4f}")
print(f"Precision: {dt_precision:.4f}")
print(f"Recall: {dt_recall:.4f}")
print(f"F1-Score: {dt_f1:.4f}")
print(f"AUC-ROC: {dt_auc_roc:.4f}")

print("  ")
print("Random Forest (Untuned) Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(rf_accuracy_scores)/len(rf_accuracy_scores):.4f}")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall: {rf_recall:.4f}")
print(f"F1-Score: {rf_f1:.4f}")
print(f"AUC-ROC: {rf_auc_roc:.4f}")

print("  ")
print("Random Forest (Tuned v1) Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(rf2_accuracy_scores)/len(rf2_accuracy_scores):.4f}")
print(f"Precision: {rf2_precision:.4f}")
print(f"Recall: {rf2_recall:.4f}")
print(f"F1-Score: {rf2_f1:.4f}")
print(f"AUC-ROC: {rf2_auc_roc:.4f}")

print( "  ")
print("Random Forest (Tuned v2) Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(rf3_accuracy_scores)/len(rf3_accuracy_scores):.4f}")
print(f"Precision: {rf3_precision:.4f}")
print(f"Recall: {rf3_recall:.4f}")
print(f"F1-Score: {rf3_f1:.4f}")
print(f"AUC-ROC: {rf3_auc_roc:.4f}")

print("  ")
print("XGBoost (Untuned) Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(xgb_accuracy_scores)/len(xgb_accuracy_scores):.4f}")
print(f"Precision: {xgb_precision:.4f}")
print(f"Recall: {xgb_recall:.4f}")
print(f"F1-Score: {xgb_f1:.4f}")
print(f"AUC-ROC: {xgb_auc_roc:.4f}")

print("  ")
print("XGBoost (Tuned) Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(xgb2_accuracy_scores)/len(xgb2_accuracy_scores):.4f}")
print(f"Precision: {xgb2_precision:.4f}")
print(f"Recall: {xgb2_recall:.4f}")
print(f"F1-Score: {xgb2_f1:.4f}")
print(f"AUC-ROC: {xgb2_auc_roc:.4f}")

print("  ")
print("KNN Model Evaluation Metrics:")
print(f"Average Accuracy: {sum(knn_accuracy_scores)/len(xgb_accuracy_scores):.4f}")
print(f"Precision: {knn_precision:.4f}")
print(f"Recall: {knn_recall:.4f}")
print(f"F1-Score: {knn_f1:.4f}")
print(f"AUC-ROC: {knn_auc_roc:.4f}")